## Calibration data
HCPS106
9.057162325863112429e+00 5.895833333333332149e+01
4.758888552392489757e+00 9.490600000000000591e+01

HCPS107
6.108090170278638098e+00 7.360349999999999682e+01
8.702237809597521689e+00 1.088159166666666806e+02

HCPS108
1.044536621433542045e+01 2.529833333333333556e+01
7.093747390396659469e+00 2.810266666666666424e+01

HCPS109
5.257291626377877058e+00 4.120066666666666322e+01
6.896564494295105874e+00 4.443458333333333599e+01

HCPS110
5.128994305657605501e+00 6.205908333333334070e+01
3.073897226304188024e+01 3.860908333333333786e+01

HCPS111
5.381027409530633854e+00 7.263150000000000261e+01
1.344333840917233935e+01 2.146666666666666856e+01

HCPS112
7.409197495183044602e+00 2.361412499999999852e+02
7.154978805394989649e+00 1.400794999999999959e+02

HCPS113
4.674894639556376852e+00 4.625458333333332916e+01
9.138885397412199296e+00 5.814908333333331569e+01

HCPS115
5.873704298915226829e+00 1.750487500000000125e+02
5.459166331860184407e+00 9.494433333333331859e+01

In [9]:
# Setup
import numpy as np
from numpy import linalg as LA
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal, fft
import math as math
import os
from matplotlib.patches import Polygon
import copy
from scipy.stats import norm
import statistics

def fun(f):
    if f.endswith('.npz') and not f.endswith('rst0.npz') and not f.endswith('rst1.npz') and not f.endswith('rst2.npz'):
        return True
    else:
        return False

#eliminate the first 2 trials
def fun2(f):
    if f.endswith('.npz') and not f.endswith('rst0.npz') and not f.endswith('rst1.npz') and not f.endswith('rst2.npz') and not f.endswith('a0.npz') and not f.endswith('a1.npz'):
        return True
    else:
        return False

#only keep the last 4 trials
def fun4(f):
    if  f.endswith('a6.npz') or f.endswith('a7.npz') or f.endswith('a8.npz') or f.endswith('a9.npz'):
        return True
    else:
        return False

            


In [10]:
# import all data from git
def COLLECTDATA(path0,path1,path2,path3,path4,path5,path6):
  files_0 = os.listdir(path0) 
  files_0 = list(filter(fun, files_0))
  files_1 = os.listdir(path1) 
  files_1 = list(filter(fun, files_1))
  files_2 = os.listdir(path2)
  files_2 = list(filter(fun, files_2))
  files_3 = os.listdir(path3)
  files_3 = list(filter(fun, files_3))
  files_4 = os.listdir(path4)
  files_4 = list(filter(fun, files_4))
  files_5 = os.listdir(path5)
  files_5 = list(filter(fun, files_5))
  files_6 = os.listdir(path6)
  files_6 = list(filter(fun, files_6))

  files0 = files_0
  files1 = files_1
  files2 = files_2
  files3 = files_3
  files4 = files_4
  files5 = files_5
  files6 = files_6

  SLIDER = {}; EMG = {}; FUSION50 = {}; FUSION25 = {}; FUSION75 = {}; SLIDER2 = {}; EMG2 = {}
  i = 0
  for trials in files0:
      #make a dict for each trial in the folder
      trial = dict(np.load(os.path.join(path0, trials), allow_pickle=True))
      del trial['filename'] #delete "filename"
      for k in range(len(trial.keys())): #eliminate first 5 seconds of data
        trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
      #create a nested dict, including all trials
      SLIDER['trial'+str(i)] = trial
      i+=1

  i = 0
  for trials in files1:
      trial = dict(np.load(os.path.join(path1, trials), allow_pickle=True))
      del trial['filename']
      for k in range(len(trial.keys())):
        trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
      EMG['trial'+str(i)] = trial
      i+=1

  i = 0
  for trials in files2:
      trial = dict(np.load(os.path.join(path2, trials), allow_pickle=True))
      del trial['filename']
      for k in range(len(trial.keys())):
        trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
      FUSION50['trial'+str(i)] = trial
      i+=1

  i = 0
  for trials in files3:
      trial = dict(np.load(os.path.join(path3, trials), allow_pickle=True))
      del trial['filename']
      for k in range(len(trial.keys())):
        trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
      FUSION25['trial'+str(i)] = trial
      i+=1

  i = 0
  for trials in files4:
      trial = dict(np.load(os.path.join(path4, trials), allow_pickle=True))
      del trial['filename']
      for k in range(len(trial.keys())):
        trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
      FUSION75['trial'+str(i)] = trial
      i+=1

  i = 0
  for trials in files5:
      trial = dict(np.load(os.path.join(path5, trials), allow_pickle=True))
      del trial['filename']
      for k in range(len(trial.keys())):
        trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
      SLIDER2['trial'+str(i)] = trial
      i+=1

  i = 0
  for trials in files6:
      trial = dict(np.load(os.path.join(path6, trials), allow_pickle=True))
      del trial['filename']
      for k in range(len(trial.keys())):
        trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
      EMG2['trial'+str(i)] = trial
      i+=1

  #create nested dict
  HCPS = {  'condition0': SLIDER, 
            'condition1': EMG,
            'condition2': FUSION50,
            'condition3': FUSION25,
            'condition4': FUSION75,
            'condition5': SLIDER2, 
            'condition6': EMG2}
  return HCPS

In [11]:
# #import all data from git
# def COLLECTDATA(path0,path1,path2,path3,path4,path5,path6):
#   files_0 = os.listdir(path0) 
#   files_0 = list(filter(fun, files_0))
#   files_1 = os.listdir(path1) 
#   files_1 = list(filter(fun, files_1))
#   files_2 = os.listdir(path2)
#   files_2 = list(filter(fun, files_2))
#   files_3 = os.listdir(path3)
#   files_3 = list(filter(fun, files_3))
#   files_4 = os.listdir(path4)
#   files_4 = list(filter(fun, files_4))
#   files_5 = os.listdir(path5)
#   files_5 = list(filter(fun, files_5))
#   files_6 = os.listdir(path6)
#   files_6 = list(filter(fun, files_6))

#   files0 = files_0
#   files1 = files_1
#   files2 = files_2
#   files3 = files_3
#   files4 = files_4
#   files5 = files_5
#   files6 = files_6

#   SLIDER = {}; EMG = {}; FUSION50 = {}; FUSION25 = {}; FUSION75 = {}; SLIDER2 = {}; EMG2 = {}
#   i = 0
#   for trials in files0:
#       #make a dict for each trial in the folder
#       trial = dict(np.load(os.path.join(path0, trials), allow_pickle=True))
#       #delete "filename"
#       del trial['filename']
#       #eliminate first 5 seconds of data
#       for k in range(len(trial.keys())):
#         trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#       #create a nested dict, including all trials
#       SLIDER['trial'+str(i)] = trial
#       i+=1
    
#   for trials in files5:
#       trial = dict(np.load(os.path.join(path5, trials), allow_pickle=True))
#       del trial['filename']
#       for k in range(len(trial.keys())):
#         trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#       SLIDER['trial'+str(i)] = trial
#       i+=1

#   i = 0
#   for trials in files1:
#       trial = dict(np.load(os.path.join(path1, trials), allow_pickle=True))
#       del trial['filename']
#       for k in range(len(trial.keys())):
#         trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#       EMG['trial'+str(i)] = trial
#       i+=1

#   for trials in files6:
#       trial = dict(np.load(os.path.join(path6, trials), allow_pickle=True))
#       del trial['filename']
#       for k in range(len(trial.keys())):
#         trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#       EMG['trial'+str(i)] = trial
#       i+=1

#   i = 0
#   for trials in files2:
#       trial = dict(np.load(os.path.join(path2, trials), allow_pickle=True))
#       del trial['filename']
#       for k in range(len(trial.keys())):
#         trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#       FUSION50['trial'+str(i)] = trial
#       i+=1

#   i = 0
#   for trials in files3:
#       trial = dict(np.load(os.path.join(path3, trials), allow_pickle=True))
#       del trial['filename']
#       for k in range(len(trial.keys())):
#         trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#       FUSION25['trial'+str(i)] = trial
#       i+=1

#   i = 0
#   for trials in files4:
#       trial = dict(np.load(os.path.join(path4, trials), allow_pickle=True))
#       del trial['filename']
#       for k in range(len(trial.keys())):
#         trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#       FUSION75['trial'+str(i)] = trial
#       i+=1

#   # i = 0
#   # for trials in files5:
#   #     trial = dict(np.load(os.path.join(path5, trials), allow_pickle=True))
#   #     del trial['filename']
#   #     for k in range(len(trial.keys())):
#   #       trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#   #     SLIDER2['trial'+str(i)] = trial
#   #     i+=1

#   # i = 0
#   # for trials in files6:
#   #     trial = dict(np.load(os.path.join(path6, trials), allow_pickle=True))
#   #     del trial['filename']
#   #     for k in range(len(trial.keys())):
#   #       trial[list(trial.keys())[k]] = trial[list(trial.keys())[k]][301:-1]
#   #     EMG2['trial'+str(i)] = trial
#   #     i+=1
#   SLIDER2 = SLIDER
#   EMG2 = EMG

#   #create nested dict
#   HCPS = {  'condition0': SLIDER, 
#             'condition1': EMG,
#             'condition2': FUSION50,
#             'condition3': FUSION25,
#             'condition4': FUSION75,
#             'condition5': SLIDER2, 
#             'condition6': EMG2}
#   return HCPS

In [12]:
HCPS = {}
participant_list = ['HCPS101','HCPS102','HCPS103','HCPS104','HCPS105','HCPS106','HCPS107','HCPS108','HCPS109','HCPS110','HCPS111','HCPS112','HCPS113','HCPS114','HCPS115']
participants_num = len(participant_list) # number of participants/subjects
k = 0
for i in participant_list:
    path0 = 'C:/Users/amber/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_slider_ctl'
    path1 = 'C:/Users/amber/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_emg_ctl'
    path2 = 'C:/Users/amber/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_fusion_emg50'
    path3 = 'C:/Users/amber/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_fusion_emg25'
    path4 = 'C:/Users/amber/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_fusion_emg75'
    path5 = 'C:/Users/amber/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_slider_ctl2'
    path6 = 'C:/Users/amber/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_emg_ctl2'
    HCPS['subject'+str(k)]= COLLECTDATA(path0,path1,path2,path3,path4,path5,path6)
    k += 1

# #home
# for i in participant_list:
#     path0 = 'C:/Users/hachou/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_slider_ctl'
#     path1 = 'C:/Users/hachou/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_emg_ctl'
#     path2 = 'C:/Users/hachou/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_fusion_emg50'
#     path3 = 'C:/Users/hachou/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_fusion_emg25'
#     path4 = 'C:/Users/hachou/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_fusion_emg75'
#     path5 = 'C:/Users/hachou/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_slider_ctl2'
#     path6 = 'C:/Users/hachou/Documents/GitHub/sensorfusion/data/'+i+'/'+i+'_emg_ctl2'
#     HCPS['subject'+str(k)]= COLLECTDATA(path0,path1,path2,path3,path4,path5,path6)
#     k += 1

In [13]:
import pickle
a_file = open("HCPS.pkl", "wb")
pickle.dump(HCPS, a_file)
a_file.close()